# Training Models

## Linear Regression

Equation of Linear Regression model prediction

$ y = \theta_{0} + \theta_{1}x_{1} + \theta_{2}x_{2} + ... + \theta_{n}x_{n} $

- y is the predicted value.
- n is the number of features
- $x_{i}$ is the $i^{th}$ feature value
- $\theta_{j}$ is the $j^{th}$ model parameter (including the bias term $\theta_{0}$ and the feature weights $\theta_{1},\theta_{2},...,\theta_{n}$)

This can be written much more concisely using a vectorized form:

$y = h_{\theta}(x) = \theta · x$ 

- $\theta$ is the model's parameter vector, containing the bias term $\theta_{0}$ and the feature weights $\theta_{1}$ to $\theta_{n}$
- x is the instance's feature vector, containing $x_{0}$ to $x_{n}$, with $x_{0}$ always equal to 1